# SRT 매크로 만들기 ( + PyKakao )
- 카카오 로그인
- 카카오로 너한테 메세지 보내도 되는지 auth 관련 code 받기
  
- SRT에 들어가요!
- 로그인을 합니다.
- 예매 페이지로 갑니다
- 원하는 출발지/도착지/시간 을 입력 시킵니다.
- 매진이 없어질때 까지 광클 합니다!

- 예약을 성공적으로 누르면 카카오톡에 메세지를 보냅니다
  

#### 환경세팅

In [12]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [25]:
browser = webdriver.Chrome()
wait = WebDriverWait(browser, 10)

## PyKakao 사용

#### 카카오 로그인

In [26]:
from PyKakao import Message

api_key = 'Kakao API KEY'

API = Message(service_key = api_key)
auth_url = API.get_url_for_generating_code()

browser.get(auth_url)

In [27]:
kakao_id = input("카카오 아이디 입력 : ")
kakao_pw = getpass.getpass("카카오 pw입력: ")

카카오 아이디 입력 :  01066282770
카카오 pw입력:  ········


In [28]:
kakao_id_input = wait.until(EC.presence_of_element_located((By.ID, 'loginId--1')))
kakao_pw_input = wait.until(EC.presence_of_element_located((By.ID, 'password--2')))
kakao_submit_btn = browser.find_element(By.CLASS_NAME, 'btn_g')

kakao_id_input.clear()
kakao_pw_input.clear()

kakao_id_input.send_keys(kakao_id)
kakao_pw_input.send_keys(kakao_pw)
kakao_submit_btn.click()

print("카카오 앱에서 확인을 눌러주세요!")

카카오 앱에서 확인을 눌러주세요!


#### 보안 관련 code 받기!!

In [29]:
kakao_agree_btn = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'btn_agree')))
kakao_agree_btn.click()

In [30]:
import re

code_url = browser.current_url
print(code_url)

code = re.search(r"code=(.+)", str(code_url))
code = code.group(1)
print(code)

https://localhost:5000/?code=ImVT0TkmkqG50D6xlnsOmlcKWP2nh1Xgr9bMVey2qDD1fcfmmtgYAD28Y44KPXKYAAABjsdr8m_kNSpXBP-m7Q
ImVT0TkmkqG50D6xlnsOmlcKWP2nh1Xgr9bMVey2qDD1fcfmmtgYAD28Y44KPXKYAAABjsdr8m_kNSpXBP-m7Q


## selenium 관련 코드

#### SRT에 들어가요!

In [31]:
url = 'https://etk.srail.kr/cmc/01/selectLoginForm.do?pageId=TK0701000000' # SRT 로그인 페이지 URL

browser.get(url)

#### 로그인을 합니다

In [32]:
import getpass

email = input("ID(이메일)를 입력해 주세요 : ")
pw = getpass.getpass("PW를 입력해 주세요 : ")

ID(이메일)를 입력해 주세요 :  tjsgh531@naver.com
PW를 입력해 주세요 :  ········


In [33]:
email_login_btn = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="srchDvCd2"]')))
email_login_btn.click()

id_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="srchDvNm02"]')))
pw_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="hmpgPwdCphd02"]')))
submit_btn = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="login-form"]/fieldset/div[1]/div[1]/div[3]/div/div[2]/input')))

id_input.clear()
pw_input.clear()

id_input.send_keys(email)
pw_input.send_keys(pw)
submit_btn.click()

#### 예매 페이지로 갑니다

In [34]:
booking_url = 'https://etk.srail.kr/hpg/hra/01/selectScheduleList.do?pageId=TK0101010000'
browser.get(booking_url)

#### 원하는 출발지/도착지/출발시간을 입력받습니다.

In [35]:
departure = input("어디서 출발하나요? : ")
destination = input("어디로 갈까요? : ") 

어디서 출발하나요? :  부산
어디로 갈까요? :  동탄


In [36]:
departure_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dptRsStnCdNm"]')))
destination_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="arvRsStnCdNm"]')))

departure_input.clear()
destination_input.clear()

departure_input.send_keys(departure)
destination_input.send_keys(destination, Keys.ENTER)

In [37]:
time_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dptTm"]')))
times = time_input.find_elements(By.TAG_NAME, 'option')
search_btn = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="search_top_tag"]/input')))

for time in times:
    print(f"{time.text}시", end=" / ")
wanted_time = int(input("가 있습니다. 몇 시로 예매 할까요? : ")) 

for time in times:
    if wanted_time == int(time.text):
        time.click()
        break

search_btn.click()

00시 / 02시 / 04시 / 06시 / 08시 / 10시 / 12시 / 14시 / 16시 / 18시 / 20시 / 22시 / 

가 있습니다. 몇 시로 예매 할까요? :  22


#### 매진이 아닐때 까지 광클을 합니다

In [38]:
import time

while True:
    special_seat = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="result-form"]/fieldset/div[6]/table/tbody/tr[1]/td[6]')))
    economy_seat = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="result-form"]/fieldset/div[6]/table/tbody/tr[1]/td[7]')))
    
    is_special_seat = False
    is_economy_seat = False

    if '예약 하기' in special_seat.text:
        is_special_seat = True
    if '예약 하기' in economy_seat.text:
        is_economy_seat = True

    if is_special_seat:
        print("특실 자리를 예매할 수 있어요!!")
        special_seat_btn = special_seat.find_element(By.TAG_NAME, 'a')
        special_seat_btn.click()
        break
    if is_economy_seat:
        print("일반실 자리를 예매할 수 있어요!!")
        economy_seat_btn = economy_seat.find_element(By.TAG_NAME, 'a')
        economy_seat_btn.click()
        break
    print("예약 할 수 있는 표가 없어요.. 광클 합니다")
    browser.refresh()
    time.sleep(3)
    search_btn = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'btn_large ')))
    search_btn.click()

특실 자리를 예매할 수 있어요!!


#### 자리가 있으면 카톡으로 메세지를 보내자!

In [39]:
import requests

url = 'https://kauth.kakao.com/oauth/token'
redirect_uri = 'https://localhost:5000'

data = {
    'grant_type' : 'authorization_code',
    'client_id' : api_key,
    'redirect_uri' : redirect_uri,
    'code' : code,
}

response = requests.post(url, data=data)
tokens = response.json()

In [41]:
import json

booking_url = browser.current_url
url="https://kapi.kakao.com/v2/api/talk/memo/default/send"

# kapi.kakao.com/v2/api/talk/memo/default/send 

headers={
    "Authorization" : "Bearer " + tokens["access_token"]
}

data={
    "template_object": json.dumps({
        "object_type":"text",
        "text":f"예약에 성공했습니다! 10분 내로 결제하러 가세요!! {booking_url}",
        "link":{
            "web_url":"www.naver.com"
        }
    })
}

response = requests.post(url, headers=headers, data=data)
response.status_code
print(response.status_code)
if response.json().get('result_code') == 0:
	print('메시지를 성공적으로 보냈습니다.')
else:
	print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(response.json()))

200
메시지를 성공적으로 보냈습니다.
